In [11]:
import pandas as pd

zerotopic_result = pd.read_csv('musinsa_crawling_dataset/zerotopic_result.csv')

zerotopic_result.columns = ['review_ID', 'TOPIC1','TOPIC2','TOPIC3','TOPIC4','TOPIC5']

zerotopic_result

,review_ID,TOPIC1,TOPIC2,TOPIC3,TOPIC4,TOPIC5
0,review_05442,{'사이즈': 59.8,'재질': 69.95,'디자인': 48.74,'만족도': 44.77,'두께감': 57.33}
1,review_04297,{'사이즈': 64.52,'재질': 95.24,'디자인': 55.34,'만족도': 97.33,'두께감': 13.21}
2,review_06211,{'사이즈': 39.77,'재질': 76.95,'디자인': 56.74,'만족도': 48.33,'두께감': 60.72}
3,review_18408,{'사이즈': 12.37,'재질': 80.51,'디자인': 45.77,'만족도': 96.6,'두께감': 25.27}
4,review_15043,{'사이즈': 82.15,'재질': 69.49,'디자인': 59.54,'만족도': 68.18,'두께감': 81.37}
...,...,...,...,...,...,...
25872,review_12843,{'사이즈': 41.62,'재질': 57.9,'디자인': 48.25,'만족도': 56.12,'두께감': 84.91}
25873,review_12823,{'사이즈': 33.83,'재질': 30.62,'디자인': 48.01,'만족도': 73.14,'두께감': 14.72}
25874,review_06171,{'사이즈': 95.38,'재질': 69.81,'디자인': 53.86,'만족도': 81.4,'두께감': 89.11}
25875,review_22120,{'사이즈': 74.1,'재질': 70.77,'디자인': 55.66,'만족도': 90.69,'두께감': 95.35}


In [13]:
import pandas as pd
import json

# CSV 파일 읽기
df = pd.read_csv('musinsa_crawling_dataset/zerotopic_result.csv')

# 컬럼 이름 설정
df.columns = ['review_ID', 'TOPIC1','TOPIC2','TOPIC3','TOPIC4','TOPIC5']

# 각 TOPIC 컬럼을 결합하여 하나의 JSON 문자열로 만듦
df['topic'] = df[['TOPIC1', 'TOPIC2', 'TOPIC3', 'TOPIC4', 'TOPIC5']].agg(', '.join, axis=1)

# 작은따옴표를 큰따옴표로 변환하여 JSON 호환성 유지
df['topic'] = df['topic'].apply(lambda x: x.replace("'", '"'))

# JSON 문자열을 실제 Python 딕셔너리로 변환
df['topic'] = df['topic'].apply(lambda x: json.loads(x))

# 필요 없는 TOPIC 컬럼 삭제
df = df.drop(columns=['TOPIC1', 'TOPIC2', 'TOPIC3', 'TOPIC4', 'TOPIC5'])

# 결과 확인
print(df.head())

# 전처리된 데이터를 동일한 이름의 CSV 파일로 저장
df.to_csv('musinsa_crawling_dataset/zerotopic_result.csv', index=False)

print("파일이 성공적으로 저장되었습니다.")


      review_ID                                              topic
0  review_05442  {'사이즈': 59.8, '재질': 69.95, '디자인': 48.74, '만족도'...
1  review_04297  {'사이즈': 64.52, '재질': 95.24, '디자인': 55.34, '만족도...
2  review_06211  {'사이즈': 39.77, '재질': 76.95, '디자인': 56.74, '만족도...
3  review_18408  {'사이즈': 12.37, '재질': 80.51, '디자인': 45.77, '만족도...
4  review_15043  {'사이즈': 82.15, '재질': 69.49, '디자인': 59.54, '만족도...
파일이 성공적으로 저장되었습니다.


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25877 entries, 0 to 25876
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_ID  25877 non-null  object
 1   topic      25877 non-null  object
dtypes: object(2)
memory usage: 404.5+ KB


In [15]:
import pandas as pd
import ast

# 첫 번째 파일 불러오기 (기존 데이터)
df1 = pd.read_csv('musinsa_crawling_dataset/dataset_product_reivew.csv')

# 두 번째 파일 불러오기 (새로운 topic 데이터)
df2 = pd.read_csv('musinsa_crawling_dataset/zerotopic_result.csv')

# 두 번째 파일에서 'topic' 컬럼을 딕셔너리로 변환
df2['topic'] = df2['topic'].apply(ast.literal_eval)

# 첫 번째 파일과 두 번째 파일을 review_ID를 기준으로 병합 (left join)
merged_df = pd.merge(df1, df2[['review_ID', 'topic']], on='review_ID', how='left')

# 기존 topic 컬럼이 NaN인 경우에만 새로운 topic 값을 덮어쓰기
merged_df['topic_x'] = merged_df['topic_x'].combine_first(merged_df['topic_y'])

# 병합 후 불필요한 컬럼 제거
merged_df = merged_df.drop(columns=['topic_y'])
merged_df = merged_df.rename(columns={'topic_x': 'topic'})

# 최종 결과 확인
print(merged_df.head())

# 병합된 데이터를 동일한 이름으로 저장
merged_df.to_csv('musinsa_crawling_dataset/dataset_product_reivew2.csv', index=False)

print("파일이 성공적으로 저장되었습니다.")

                                              review     review_ID      date  \
0                    마음에 들어요 깔금하구 더위에 입을만 합니드 ㅎ자주입을듯  review_07226  20240628   
1  단독으로 입기에는 핏이 애매해요 원래 맨투맨이나 후드티 안에 입으려고 샀어요 부들부...  review_05442  20240627   
2                      길이며 재질이며 너무 만족해서 더 구매할 예정입니다!  review_04297  20240613   
3  딱 베이직한 핏의 티셔츠 루즈핏 아니에요 기본티나 이너티로 입기 좋아요 저렴하게 잘...  review_06211  20240523   
4  원하던 기본티라 만족합니더 그리고 재질도 짱짱해서 오래 입을 수 있을 거 같아요! ...  review_18408  20240514   

   grade   size  sex  height  weight  helpful  style_like  ...  \
0      5   흰색/M  0.0   184.0    82.0        0           0  ...   
1      5  흰색/XS  0.0   167.0    54.0        0           0  ...   
2      5   흰색/L  0.0   158.0    55.0        0           0  ...   
3      5   흰색/M  0.0   167.0    64.0        0           0  ...   
4      5   흰색/S  0.0   166.0    56.0        0           0  ...   

  reviewtag_color reviewtag_thickness reviewtag_warmth reviewtag_weight  \
0         색감 선명해요           두께감 보통이에요          

In [18]:
df = pd.read_csv('musinsa_crawling_dataset/dataset_product_reivew2.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25878 entries, 0 to 25877
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   review                25878 non-null  object 
 1   review_ID             25878 non-null  object 
 2   date                  25878 non-null  int64  
 3   grade                 25878 non-null  int64  
 4   size                  25878 non-null  object 
 5   sex                   25878 non-null  float64
 6   height                25878 non-null  float64
 7   weight                25878 non-null  float64
 8   helpful               25878 non-null  int64  
 9   style_like            25878 non-null  int64  
 10  reviewtag_size        25878 non-null  object 
 11  reviewtag_brightness  25878 non-null  object 
 12  reviewtag_color       25878 non-null  object 
 13  reviewtag_thickness   25878 non-null  object 
 14  reviewtag_warmth      25878 non-null  object 
 15  reviewtag_weight   